In [1]:
import geopandas as gpd

# Notebook overview
This notebook takes the output polygons from Tile2Net and filters them to only keep the polygons of interest. Firstly, we filter out all polygons that are not labeled as crosswalks. Secondly, we fiter out the polygons that do not fall into our area of interest which is specified with another polygon. We only keep crosswalk polygons that overlap with the area of interest polygon.

**Input**: 
- Shapely file with polygons as put out by Tile2Net.
- GeoJSON file with polygon of area of interest.

**Output**: shapely file of crosswalk polygons in correct area.

## Settings

In [2]:
# Set CRS
CRS = 'epsg:28992'

## Load data

We first load the polygons as provided by Tile2Net. We then filter for crosswalk polygons as the tool provides more than just those.

In [3]:
# Set path where T2N output polygons can be found
T2N_path = "../data/input/polygons Tile2Net/output polygons T2N.shp"

# Load polygons as put out by T2N
raw_polygons = gpd.read_file(T2N_path)

# Adjust CRS
raw_polygons = raw_polygons.to_crs(CRS)

# Filter for crosswalks polygons
CW_polygons = raw_polygons[raw_polygons['f_type'] == 'crosswalk']

In [4]:
# Set path to polygon of area
polygon_path = "../data/input/polygon area/polygon area.geojson"

# Get polygon
area_gdf = gpd.read_file(polygon_path)
area_polygon = area_gdf.loc[0]['geometry']

We only want the crosswalks that are in our area of interest. T2N has generated crosswalks for a larger area so we need to filter out the polygons that are not in our area of interest.

In [5]:
# Only keep crosswalk polygons that fall within the bouds of the area
indexes = []

# Loop over crosswalk polygons
for CW in CW_polygons.itertuples():

    # Save indices of polygons that are not in the area of interest
    if not (area_polygon.intersects(CW[2])):
        indexes.append(CW[0])

# Drop rows with polygons outside of the area of interest based on index
CW_polygons = CW_polygons.drop(index=indexes)
CW_polygons = CW_polygons.reset_index(drop=True)

In [6]:
# Save the filtered polygons
CW_polygons_path = "../data/output/crosswalk polygons Tile2Net.shp"

CW_polygons.to_file(CW_polygons_path)